In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from nltk import word_tokenize, ngrams 
from sklearn import ensemble
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
import xgboost as xgb

eng_stopwords = set(stopwords.words('english'))

from pandas import DataFrame

E:\Anaconda2\envs\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# *Prepare data*

In [2]:
train1 = pd.read_csv("train/train1.csv")
test1 = pd.read_csv("test/test1.csv")
print(train1.shape)
print(test1.shape)

train2 = pd.read_csv("train/train2.csv")
test2 = pd.read_csv("test/test2.csv")
print(train2.shape)
print(test2.shape)

train2.head()

(404290, 8)
(2345796, 5)
(404290, 7)
(2345796, 7)


,common_unigrams_len,common_unigrams_ratio,common_bigrams_len,common_bigrams_ratio,common_trigrams_len,common_trigrams_ratio,id
0,6.0,0.857143,5.0,0.625000,4.0,0.571429,0
1,6.0,0.461538,4.0,0.307692,3.0,0.250000,1
2,3.0,0.300000,0.0,0.000000,0.0,0.000000,2
3,1.0,0.062500,0.0,0.000000,0.0,0.000000,3
4,3.0,0.200000,0.0,0.000000,0.0,0.000000,4


In [4]:
train1.head()

,id,qid1,qid2,question1,question2,is_duplicate,word_match,tfidf_word_match
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,0.727273,0.772164
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,0.307692,0.361758
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,0.363636,0.355191
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,0.000000,0.000000
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,0.000000,0.000000


In [6]:
train1['xxx']=train1.qid1-train1.qid2
train1.head()

,id,qid1,qid2,question1,question2,is_duplicate,word_match,tfidf_word_match,xxx
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,0.727273,0.772164,-1
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,0.307692,0.361758,-1
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,0.363636,0.355191,-1
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,0.000000,0.000000,-1
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,0.000000,0.000000,-1


In [7]:
train1[(train1["xxx"]!=-1)]#184930 rows × 9 columns

,id,qid1,qid2,question1,question2,is_duplicate,word_match,tfidf_word_match,xxx
264,264,81,529,Why do Slavs squat?,Do squats work for men?,0,0.000000,0.000000,-448
326,326,652,108,What are the most interesting foods you have e...,What is the most delicious dish you've ever ea...,1,0.200000,0.233642,544
353,353,705,134,What are the most important books ever written?,What is the most important book you have ever ...,1,0.500000,0.433054,571
647,647,1292,1127,How do I make money with YouTube?,How do I make money through YouTube?,1,1.000000,1.000000,165
704,704,1189,1405,When will the best time for having sex?,What would be the best time to have sex?,1,0.857143,0.909044,-216
729,729,883,1454,Why is Saltwater taffy candy imported in China?,Why is Saltwater taffy candy imported in Laos?,1,0.800000,0.799245,-571
1027,1027,2049,1919,What food should I eat to gain weight?,How could I gain weight in a healthy way?,0,0.222222,0.236910,130
1081,1081,1863,2156,Did Trump win the election?,What do you think of Trump winning the elections?,0,0.285714,0.219689,-293
1114,1114,2221,1093,How can I hack a WhatsApp account?,How can I hack another WhatsApp account?,1,0.857143,0.855327,1128
1332,1332,2656,1215,Is a third world war coming?,Is World War 3 more imminent than expected?,1,0.444444,0.365184,1441


In [31]:
#train2.drop(['question1','question2'],axis=1, inplace=True)
#test2.drop(['question1','question2'],axis=1, inplace=True)

In [33]:
train_df=pd.merge(train1,train2,how='inner', on='id') 
test_df=pd.merge(test1,test2,how='inner', on='test_id') 

del train1,train2,test1,test2

print("train_df:",train_df.shape)
print("test_df",test_df.shape)
train_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,word_match,tfidf_word_match,common_unigrams_len,common_unigrams_ratio,common_bigrams_len,common_bigrams_ratio,common_trigrams_len,common_trigrams_ratio
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,0.727273,0.772164,6.0,0.857143,5.0,0.625000,4.0,0.571429
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,0.307692,0.361758,6.0,0.461538,4.0,0.307692,3.0,0.250000
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,0.363636,0.355191,3.0,0.300000,0.0,0.000000,0.0,0.000000
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,0.000000,0.000000,1.0,0.062500,0.0,0.000000,0.0,0.000000
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,0.000000,0.000000,3.0,0.200000,0.0,0.000000,0.0,0.000000


# *Add features to the Data*

# *Rebalancing the Data*

In [34]:
y_train = train_df['is_duplicate'].values
pos_train = train_df[y_train == 1]
neg_train = train_df[y_train == 0]


# Now we oversample the negative class
# There is likely a much more elegant way to do this...
p = 0.165
scale = ((len(pos_train) / (len(pos_train) + len(neg_train))) / p) - 1
while scale > 1:
    neg_train = pd.concat([neg_train, neg_train])
    scale -=1
neg_train = pd.concat([neg_train, neg_train[:int(scale * len(neg_train))]])
print(len(pos_train) / (len(pos_train) + len(neg_train)))

x_train = pd.concat([pos_train, neg_train])
y_train = (np.zeros(len(pos_train)) + 1).tolist() + np.zeros(len(neg_train)).tolist()
del pos_train, neg_train

print("train_df:",train_df.shape)
print("test_df",test_df.shape)

print("x_train:",x_train.shape)
print("y_train",len(y_train))

0.19124366100096607
train_df: (404290, 14)
test_df (2345796, 11)
x_train: (780486, 14)
y_train 780486


In [25]:
def word1(que):
    return [word for word in word_tokenize(que.lower()) if word not in eng_stopwords]#word_tokenize:对句子进行分词

def word2(que):
    return [word for word in que.lower().split() if word not in eng_stopwords]#:对句子进行分词

def get_bigrams(que):
    return [i for i in ngrams(word_tokenize(que.lower()), 2)]#组合分词

In [26]:
sentence1 = """At eight o'clock on Thursday morning Arthur didn't feel very good?"""
sentence2 = """At eight o'clock on Thursday morning Arthur feel very good!"""
x1= word1(sentence1)
x2= word1(sentence2)

y1= word2(sentence1)
y2= word2(sentence2)

z1= get_bigrams(sentence1)
z2= get_bigrams(sentence2)

print(x1)
print(x2)
print("  ")
print(y1)
print(y2)
print("  ")
print(z1)
print(z2)


['eight', "o'clock", 'thursday', 'morning', 'arthur', "n't", 'feel', 'good', '?']
['eight', "o'clock", 'thursday', 'morning', 'arthur', 'feel', 'good', '!']
  
['eight', "o'clock", 'thursday', 'morning', 'arthur', "didn't", 'feel', 'good?']
['eight', "o'clock", 'thursday', 'morning', 'arthur', 'feel', 'good!']
  
[('at', 'eight'), ('eight', "o'clock"), ("o'clock", 'on'), ('on', 'thursday'), ('thursday', 'morning'), ('morning', 'arthur'), ('arthur', 'did'), ('did', "n't"), ("n't", 'feel'), ('feel', 'very'), ('very', 'good'), ('good', '?')]
[('at', 'eight'), ('eight', "o'clock"), ("o'clock", 'on'), ('on', 'thursday'), ('thursday', 'morning'), ('morning', 'arthur'), ('arthur', 'feel'), ('feel', 'very'), ('very', 'good'), ('good', '!')]
